# Готовим данные для ARTM

В этом блокноте готовим данные, взятые из контакта для ARTM. Библиотека поддерживает [конкретные виды входа.](http://docs.bigartm.org/en/stable/tutorials/datasets.html) Будем перегонять их в **vowpal wabbit** формат, как наиболее хайповый. В нашей статье всё хайповое и трендовое. Также будем чистить данные от всякой мерзости и лемматизировать. 

In [1]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами

# Пакет для красивых циклов. При желании его можно отключить. 
# Тогда из всех циклов придётся  удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

import re      # пакет для работы с регулярными выражениями 
import pickle  # пакет для подгрузки данных специфического для питона формата

# работа со временем
from datetime import datetime

# Токенизатор
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Лемматизатор (приводит слова к нормальной форме)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Список стоп-слов
from nltk.corpus import stopwords
stop_ru = stopwords.words('russian')
stop_en = stopwords.words('english')
stop = stop_ru + stop_en

# Для распараллеливания кода
from joblib import Parallel, delayed

# Для перемешивания
from random import shuffle

In [2]:
# Путь к данным
path = '../data/'

# жанры 
music = ['estrada', 'popsa', 'rock']

In [3]:
def dataLoad(music_name, path = path, data_type='group_comments'):
    """ 
        Подгружает что-нибудь эдакое
        music_name: string
            название жанра
        path: string
            путь до директории с данными
        data_type
            суффикс файла (group_comments - комментарии, users_wall, ...) 
    """
    # подгружаем данные 
    with open(path + music_name + '_' + data_type, 'rb') as f:
        df = pickle.load(f)
    return df

In [4]:
def htmlStrip(text):
    """ 
        Возвращает текст, очещенный от html тэгов
        text: string
            Текст поста 
    """
    
    return re.sub('<[^<]+?>', '', str(text)) 


def tokenLemma(text, stop = stop):
    """ 
        Возвращает список слов, очищенный от тэгов, пролемматизированный и без стоп слов
        text: string
            Текст поста
        
        parameters: list 
            stop: список из стоп-слов, example: ['политик', 'выбирать']
    """
    
    # понижение регистра, очистка от тэгов, токенизация
    out_1 = tokenizer.tokenize(htmlStrip(text.lower()))
    # лемматизация и очистка от стоп-слов, ещё выкидывает слова короче 3 символов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1 if tx not in stop and len(tx) > 2]
    return ' '.join(out_2)

## 1. Предобработка комментариев

В конечном итоге нам нужно, чтобы каждый комментарий перегонялся в формат **vowpal wabbit:**

```
'popsa_бузова_4887563_208109_208174 |@default_class id289877716 глеб публикация какой какой число день какой коммент ольга ответить это плохой комментатор один видеть олечка ответить коммент|@likes 0 |@stickers sticker_1 sticker_2  |@audio николай басков шарманка |@video клип басков шарманка |@link плейлист киркоров |@emoji 😊😊😊'
```

Модальности: 

* имя коммента `жанр исполнитель id группы_id поста_id коммента`
* текст коммента 
* число лайков
* стикеры в комменте 
* название аудиозаписи, вложенной в коммент 
* название и описание видеозаписи, вложенной в коммент 
* ссылка на плейлист или ещё куда и её описание
* эмодзи, которые были в комменте

Именно в таком виде мы будем подавать комменты в модель для обучения. Кроме такого формата будем делать прмоежуточный: обычный массив из словарей с комментами. Его можно будет быстро отфильтровать по какому-то признаку, превратить в табличку и тп. Именно его будем использовать для сентимент-модели и матерных списков Олечки. 

In [ ]:
def firstCommentPrepare(comment, style, performer):
    """
        Первичная предобработка комментария в словарик
    """
    
    # интересные вложения
    stickers = [ ]  # стикеры 
    audio = [ ]     # музыка 
    video = [ ]     # видяшки
    # photo_cnt = 0   # счётчик для фоточек с красоточками
    # doc_cnt = 0     # чаще всего это фотки либо гифки
    link = [ ]      # ссылки на плейлисты или ещё куда
        
    attach = comment.get('attachments')
    if attach is not None:
        for at in attach:
            tp = at['type']
            if tp == 'sticker':
                stickers.append(tp + '_' + str(at[tp]['sticker_id']))
            elif tp == 'audio':
                audio.append(at[tp]['artist'] + ' ' + at[tp]['title'])
            elif tp == 'video':
                # из интересных вложений есть ещё shows, comments, duration
                video.append(at[tp]['title'] + ' ' + at[tp]['description'])
            # elif tp == 'photo':
            #     photo_cnt += 1
            # elif tp == 'doc':
            #     doc_cnt += 1
            elif tp == 'link':
                link.append(at[tp]['title'] + ' ' + at[tp]['description']) 
                
    # Выбрасываем из записи все символы, кроме эмодзи и редких символов вроде иероглифов         
    emoji = re.sub('[a-яa-zA-ZА-Я -.,0-9<>:=;/_—@$^?–ё`%#*&!№…\|/\"\]\[\"\'\n\t”“’‘]','',comment['text'])
                    
    comment_infa = {     
        'music_style' : name,
        'performer' : performer,
        'comment_id' :  str(comment['group_id']) + '_' + str(comment['post_id'])  + '_' + str(comment['id']),
        'author' : str(comment['from_id']),
        'likes' : str(comment['likes']),
        'date' : datetime.utcfromtimestamp(comment['date']).strftime('%Y-%m-%d %H:%M:%S'),
        'text' : tokenLemma(comment['text']),
        'dirty_text' : comment['text'],
        'emoji' : emoji if len(emoji) > 1 else None,
        'stickers' : ' '.join(stickers) if len(stickers) > 0 else None, 
        'audio' : tokenLemma(' '.join(audio)) if len(audio) > 0 else None,
        'video' : tokenLemma(' '.join(video)) if len(video) > 0 else None,
        'link' : tokenLemma(' '.join(link)) if len(link) > 0 else None
        }
    return comment_infa


def secondCommentPrepare(comment, modality=True):
    """
    Вторичная предобработка комментариев из словаря в vw
        comment : string 
            словарик с комментарием
        modality : bool
            нужны ли модальности по видео, аудио, ссылкам и тп
    """
    # инфу про автора и дату я нигде не использую 
    comment_name = comment['music_style'] + '_' + comment['performer'] + '_' + comment['comment_id']
    likes = '|@likes {} '.format(comment['likes'])
    text = '|@default_class {}'.format(comment['text'])
    
    # базовая инфа для модели 
    comment_body = comment_name + text  + likes
    
    # дополнительные модальности 
    if modality:
        if comment['stickers']:
            comment_body = comment_body + ' |@stickers ' + comment['stickers']
        if comment['audio']:
            comment_body = comment_body + ' |@audio ' + comment['audio']
        if comment['video']:
            comment_body = comment_body + ' |@video ' + comment['video']
        if comment['link']:
            comment_body = comment_body + ' |@link ' + comment['link']
        if comment['emoji']:
            comment_body = comment_body + ' |@emoji ' + comment['emoji']
        
            
    return comment_body

In [ ]:
df = dataLoad('popsa')
df.keys()

In [63]:
test = firstCommentPrepare(df['Бузова'][10],'popsa','бузова')
test

{'audio': 'dmitriev ольга бузов танцевать бузов',
 'author': '8838081',
 'comment_id': '4887563_208109_208139',
 'date': '2018-12-31 08:54:53',
 'dirty_text': 'и тут же записан кавер 😊😊😊\nhttps://vk.com/dmitriev05',
 'emoji': '😊😊😊',
 'likes': '2',
 'link': None,
 'music_style': 'popsa',
 'performer': 'бузова',
 'stickers': None,
 'text': 'записать кавер https com dmitriev05',
 'video': None}

In [64]:
secondCommentPrepare(test)

'popsa_бузова_4887563_208109_208139|@default_class записать кавер https com dmitriev05|@likes 2 '

In [65]:
secondCommentPrepare(test, modality=True)

'popsa_бузова_4887563_208109_208139|@default_class записать кавер https com dmitriev05|@likes 2  |@audio dmitriev ольга бузов танцевать бузов |@emoji 😊😊😊'

In [66]:
comments = [ ]

for name in music:
    df = dataLoad(name)
    
    for key in tqdm_notebook(df):
        print('Предобрабатывают',name)
        # Внимание, костыль! 
        def prCom(w):
            return firstCommentPrepare(w,name,key)
        
        n_jobs = -1 # параллелим на все ядра 
        result = Parallel(n_jobs=n_jobs)(delayed(prCom)(text) for text in tqdm_notebook(df[key]))
        comments.extend(result)
        print('Текущее число комментариев:', len(comments))

HBox(children=(IntProgress(value=0, max=78098), HTML(value='')))


Текущее число комментариев: 78098


HBox(children=(IntProgress(value=0, max=112905), HTML(value='')))


Текущее число комментариев: 191003


HBox(children=(IntProgress(value=0, max=639), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=3192), HTML(value='')))


Текущее число комментариев: 445885


HBox(children=(IntProgress(value=0), HTML(value='')))


Текущее число комментариев: 445985


KeyboardInterrupt: 

In [67]:
print(len(comments))
comments[0]

445985


{'audio': None,
 'author': '134973383',
 'comment_id': '52228413_282387_282388',
 'date': '2018-12-31 21:40:35',
 'dirty_text': 'Шикарный подарок , песня замечательная !!!! Спасибоооо !! С Новым годом всех друзья!!!!',
 'emoji': None,
 'likes': '21',
 'link': None,
 'music_style': 'estrada',
 'performer': 'Лепс',
 'stickers': None,
 'text': 'шикарный подарок песня замечательный спасибоооо новый год друг',
 'video': None}

In [68]:
shuffle(comments) # перемешаем комменты
comments[0]

{'audio': None,
 'author': '116382495',
 'comment_id': '24164800_93961_93987',
 'date': '2016-02-14 15:34:27',
 'dirty_text': '[id186711409|Никита], католический праздник, иначе его называют " Днём всех влюблённых " , но к творчеству А. Пугачёвой он никакого отношения не имеет, однозначно. 😎',
 'emoji': None,
 'likes': '0',
 'link': None,
 'music_style': 'estrada',
 'performer': 'Пугачёва',
 'stickers': None,
 'text': 'id186711409 никита католический праздник иначе называть днём влюбить творчество пугачёвый никакой отношение иметь однозначно',
 'video': None}

In [ ]:
# сохраним предобработанные данные 
with open('../data/clean_comments'.format(music_style), 'wb') as f:
    pickle.dump(group_users_dct, f)

In [71]:
# отфильтруем слишком короткие комменты (меньше 10 символов) 
# тут же надо будет сделать сурьёзные махинации с отбором комментов по длине и сэмплом по жанрам 

comments = [com for com in comments if len(com['text']) > 10]
len(comments)

375031

In [73]:
n_jobs = -1 # параллелим на все ядра 
artm_result = Parallel(n_jobs=n_jobs)(delayed(secondCommentPrepare)(
    text) for text in tqdm_notebook(comments))

HBox(children=(IntProgress(value=0, max=375031), HTML(value='')))

In [74]:
artm_result[0]

'estrada_Пугачёва_24164800_93961_93987|@default_class id186711409 никита католический праздник иначе называть днём влюбить творчество пугачёвый никакой отношение иметь однозначно|@likes 0 '

In [ ]:
# сохраняем полученное добро 
with open('../data_artm/comments_v1', 'a') as out:
    for item in comments:
        out.write("{}\n".format(item))

## 2. Предобработка стенок юзеров

In [ ]:
# задел на будущее

## 3. Предобраьотка постов из групп

In [ ]:
# задел на будущее